# Worker Callback

## 介绍

Worker 回调机制将在工作者执行的关键点调用预定义的钩子方法，使您能够添加诸如日志记录、验证、监控和错误处理等跨切关注点，而无需修改您的业务逻辑。本教程将指导您理解和使用 Worker 回调机制。

`WorkerCallback` 是所有围绕工作者调用的回调实例的基类。您可以实现以下三个方法来子类化 `WorkerCallback`：

| 方法                  | 描述                                                                                   |
|:---------------------:|:---------------------------------------------------------------------------------------:|
| `on_worker_start()`   | 在工作者执行之前调用。用于输入验证、日志记录或监控                                     |
| `on_worker_end()`     | 在成功执行后调用。用于结果日志记录、事件发布或指标                                     |
| `on_worker_error()`   | 当引发异常时调用。用于错误处理、日志记录或抑制                                         |

## 创建自定义回调

### 步骤 1：定义一个类

要创建自定义回调，只需继承 `WorkerCallback` 类并实现上述方法。您不需要实现所有三个方法，只需实现您需要的方法即可。基础的 `WorkerCallback` 类提供了默认实现，这些实现不执行任何操作。这里我们定义一个 `LoggingCallback` 类，它实现了所有三个钩子方法。

In [19]:
from typing import Any, Dict, Optional
from bridgic.core.automa import GraphAutoma
from bridgic.core.automa.worker import WorkerCallback

class LoggingCallback(WorkerCallback):
    """Log worker lifecycle events."""

    def __init__(self, tag: str = None):
        self._tag = tag or ""

    async def on_worker_start(
        self,
        key: str,
        is_top_level: bool = False,
        parent: Optional[GraphAutoma] = None,
        arguments: Dict[str, Any] = None,
    ) -> None:
        print(self._tag + f"[START] {key} args={arguments}")

    async def on_worker_end(
        self,
        key: str,
        is_top_level: bool = False,
        parent: Optional[GraphAutoma] = None,
        arguments: Dict[str, Any] = None,
        result: Any = None,
    ) -> None:
        print(self._tag + f"[END] {key} result={result}")

    async def on_worker_error(
        self,
        key: str,
        is_top_level: bool = False,
        parent: Optional[GraphAutoma] = None,
        arguments: Dict[str, Any] = None,
        error: Exception = None,
    ) -> bool:
        print(self._tag + f"[ERROR] {key} -> {error}")
        return False  # Returning False means don't suppress the exception.

### 第2步：选择构建模式

回调通过 `WorkerCallbackBuilder` 实例化，该过程延迟到工作者创建时，并让您控制其共享模式。

- **共享实例模式** (`is_shared=True`, 默认)：声明范围内的所有工作者重用相同的回调实例。这对于有状态的集成（例如，跟踪客户端）是理想的。
- **独立实例模式** (`is_shared=False`)：每个工作者接收其自己的回调实例。当您需要隔离状态或线程安全时，这非常有用。

In [20]:
from bridgic.core.automa.worker import WorkerCallbackBuilder

# Build in shared instance mode.
shared_builder = WorkerCallbackBuilder(LoggingCallback)
isolated_builder = WorkerCallbackBuilder(LoggingCallback, is_shared=False)

# Build in independent instance mode.
shared_builder_with_args = WorkerCallbackBuilder(LoggingCallback, init_kwargs={"tag": "-"})
isolated_builder_with_args = WorkerCallbackBuilder(LoggingCallback, init_kwargs={"tag": "-"}, is_shared=False)

### 第三步：决定范围

选择构建器的作用范围。Bridgic 从最广泛的范围到最狭窄的范围合并构建器：

| 级别              | 注册方法                   | 作用范围                                          |
|:-----------------:|:-------------------------:|:-------------------------------------------------|
| **全局级别**      | `GlobalSetting`           | 适用于所有 Automa 实例中的所有工作者             |
| **Automa 级别**   | `RunningOptions`          | 适用于一个 Automa 实例中的每个工作者             |
| **工作者级别**    | `@worker` 或 `add_worker()` | 仅适用于目标工作者                                |

**全局级别配置**

以下示例展示了如何使用 `GlobalSetting` 在全局级别配置回调，这将回调应用于您应用中所有 Automa 实例的所有 Worker。

In [21]:
from bridgic.core.automa import GraphAutoma, worker
from bridgic.core.automa.worker import WorkerCallbackBuilder
from bridgic.core.config import GlobalSetting

GlobalSetting.set(callback_builders=[WorkerCallbackBuilder(LoggingCallback)])

class MyAutoma(GraphAutoma):
    @worker(is_start=True)
    async def step1(self, x: int) -> int:
        return x + 1

automa = MyAutoma(name="test-automa")  # Will log for all workers inside.
await automa.arun(x=10)

[START] test-automa args={'args': (), 'kwargs': {'x': 10}, 'feedback_data': None}
[START] step1 args={'args': (), 'kwargs': {'x': 10}}
[END] step1 result=11
[END] test-automa result=None


**Automa-Level Configuration**

以下示例展示了如何使用 `RunningOptions` 在 automa 级别配置回调，这将回调应用于特定 Automa 实例中的所有 worker。

In [22]:
from bridgic.core.automa import GraphAutoma, RunningOptions, worker
from bridgic.core.automa.worker import WorkerCallbackBuilder

# Because this example is under jupyter-notebook environment, GlobalSetting needs to be reset.
# In real deveopment, this line is not necessary to have a default global setting.
GlobalSetting.set(callback_builders=[])

class MyAutoma(GraphAutoma):
    @worker(is_start=True)
    async def step1(self, x: int) -> int:
        return x + 1

running_options = RunningOptions(callback_builders=[WorkerCallbackBuilder(LoggingCallback)])
automa = MyAutoma(name="test-automa", running_options=running_options)  # Will log for all workers inside.
await automa.arun(x=10)

[START] test-automa args={'args': (), 'kwargs': {'x': 10}, 'feedback_data': None}
[START] step1 args={'args': (), 'kwargs': {'x': 10}}
[END] step1 result=11
[END] test-automa result=None


**Worker-Level Configuration**

以下示例展示了如何通过将 `callback_builders` 传递给 `@worker` 装饰器在 Worker 级别配置回调，这样回调仅应用于特定的 Worker。

In [23]:
from bridgic.core.automa import GraphAutoma, worker
from bridgic.core.automa.worker import WorkerCallbackBuilder

# Because this example is under jupyter-notebook environment, GlobalSetting needs to be reset.
# In real deveopment, this line is not necessary to have a default global setting.
GlobalSetting.set(callback_builders=[])

class MyAutoma(GraphAutoma):
    @worker(
        is_start=True,
        callback_builders=[WorkerCallbackBuilder(LoggingCallback)],
    )
    async def step1(self, x: int) -> int:
        return x + 1

automa = MyAutoma(name="test-automa")  # Will only log for "step1" worker.
await automa.arun(x=10)


[START] step1 args={'args': (), 'kwargs': {'x': 10}}
[END] step1 result=11


## 你需要了解的功能

### 回调传播

嵌套的 automa 将通过读取初始化的运行选项，从其父（甚至更高的祖先）作用域继承回调。这确保了在多个层级之间的仪器保持一致。当你在 automa 初始化期间在 `RunningOptions` 中设置回调时，任何嵌套 automata 中的所有工作者将自动继承该回调：

在以下示例中，配置在顶层 automa 的 `LoggingCallback` 传播到：
- 直接在 `TopAutoma` 中的所有工作者（`top_worker`，`nested_automa_worker`）
- 嵌套的 `InnerAutoma` 中的所有工作者（`inner_worker`）

In [24]:
from bridgic.core.automa import GraphAutoma, RunningOptions, worker
from bridgic.core.automa.worker import WorkerCallback, WorkerCallbackBuilder
from bridgic.core.config import GlobalSetting

# Top-level automa
class TopAutoma(GraphAutoma):
    @worker(is_start=True)
    async def top_worker(self, x: int) -> int:
        return x + 1

# Inner automa (will be used as a nested worker)
class InnerAutoma(GraphAutoma):
    @worker(is_start=True)
    async def inner_worker(self, x: int) -> int:
        return x * 2

# Configure callback at global setting, with <Global> tag.
GlobalSetting.set(
    callback_builders=[
        WorkerCallbackBuilder(LoggingCallback, init_kwargs={"tag": "<Global>"}),
    ]
)

# Configure callback at top-level automa, with <Automa> tag.
running_options = RunningOptions(
    callback_builders=[
        WorkerCallbackBuilder(LoggingCallback, init_kwargs={"tag": "<Automa>"})
    ]
)
automa = TopAutoma(name="top-automa", running_options=running_options)

# Add a instance of InnerAutoma as a worker.
automa.add_worker("nested_automa_as_worker", InnerAutoma(name="inner-automa"), dependencies=["top_worker"])

# When executed:
# - Callbacks that from GlobalSetting will be propagated to all workers application-wide.
# - Callbacks that from RunningOptions will be propagated to all workers inside the "top-level" automa.
await automa.arun(x=10)


<Global>[START] top-automa args={'args': (), 'kwargs': {'x': 10}, 'feedback_data': None}
<Automa>[START] top-automa args={'args': (), 'kwargs': {'x': 10}, 'feedback_data': None}
<Global>[START] top_worker args={'args': (), 'kwargs': {'x': 10}}
<Automa>[START] top_worker args={'args': (), 'kwargs': {'x': 10}}
<Global>[END] top_worker result=11
<Automa>[END] top_worker result=11
<Global>[START] nested_automa_as_worker args={'args': (11,), 'kwargs': {'feedback_data': None, 'x': 10}}
<Automa>[START] nested_automa_as_worker args={'args': (11,), 'kwargs': {'feedback_data': None, 'x': 10}}
<Global>[START] inner_worker args={'args': (11,), 'kwargs': {}}
<Automa>[START] inner_worker args={'args': (11,), 'kwargs': {}}
<Global>[END] inner_worker result=22
<Automa>[END] inner_worker result=22
<Global>[END] nested_automa_as_worker result=None
<Automa>[END] nested_automa_as_worker result=None
<Global>[END] top-automa result=None
<Automa>[END] top-automa result=None


### 动态拓扑支持

`GraphAutoma.add_worker()` 和相关的 API 允许您在运行时修改拓扑。当添加新 worker 时，Bridgic 会自动使用当前全局构建器、其祖先的运行选项中的构建器以及通过 `add_worker()` 调用传递的构建器来构建其回调列表。因此：

- 动态添加的 workers 接收与静态声明的 workers 相同的仪器保证。
- 作为新 worker 后插入的嵌套 automa 继承其祖先作用域的回调。

这种设计使得长时间运行的智能系统在执行过程中即使在增长或重新配置时也保持可观测性。

### 异常处理

#### 异常类型匹配

`on_worker_error()` 方法通过检查其 `error` 参数的类型注释，允许进行细粒度和灵活的错误处理。您可以通过用特定异常类型注释 `error` 参数，明确指示您的处理程序要响应哪些异常类型。在运行时，Bridgic 框架将自动匹配并仅对与注释匹配的异常调用您的回调。

下面是一个简单的示例比较表：

| `error` 的类型注释       | 触发 `on_worker_error` 的匹配异常类型                   |
|:--------------------------:|:-------------------------------------------------------------|
| `Exception`                | 所有异常                                               |
| `ValueError`               | `ValueError` 及其子类（例如，`UnicodeDecodeError`） |
| `Union[Type1, Type2, ...]` | `Type1` 和 `Type2` 将被视为匹配                     |

In [25]:
import warnings

from typing import Union
from bridgic.core.automa.worker import WorkerCallback

class ValueErrorHandler(WorkerCallback):
    async def on_worker_error(
        self,
        key: str,
        is_top_level: bool = False,
        parent: Optional[GraphAutoma] = None,
        arguments: Dict[str, Any] = None,
        error: ValueError = None
    ) -> bool:
        warnings.warn("ValueError in %s: %s", key, error)
        return True  # Swallow ValueError.

class MultipleErrorHandler(WorkerCallback):
    async def on_worker_error(
        self,
        key: str,
        is_top_level: bool = False,
        parent: Optional[GraphAutoma] = None,
        arguments: Dict[str, Any] = None,
        error: Union[KeyError, TypeError] = None
    ) -> bool:
        warnings.warn("Recoverable issue: %s", error)
        return False  # Re-raise it.

#### 异常抑制

`on_worker_error` 的返回值将决定是否抑制捕获的异常：

| 值      | 行为                                                                      |
|:-------:|:---------------------------------------------------------------------------|
| `True`  | 意味着抑制异常；worker 结果变为 `None`。                                   |
| `False` | 意味着仅进行观察；框架在所有回调完成后重新抛出异常。                     |

特别地，为了确保人在回路的流程保持完整，`InteractionException` 不应被抑制。

框架会调用每个匹配的回调，因此您可以使用更广泛的“捕获所有”回调来组合专门的处理程序。

## 最佳实践

1. **保持回调轻量**：回调方法会为它们负责的每个工作者调用，因此保持它们快速并避免阻塞操作。

2. **使用适当的作用域**：对于应用程序范围的关注使用全局级别，对于特定实例使用automa级别，对于细粒度控制使用工作者级别。

3. **小心处理异常**：要仔细考虑要抑制哪些异常。抑制异常可能会隐藏错误并使调试变得困难。

4. **明智地使用共享实例**：共享实例非常适合维护连接或状态，但要注意线程安全问题。

5. **利用父参数**：`parent`参数使您可以访问automa的上下文，从而允许您发布事件、请求反馈或与automa的状态进行交互。

## 下一步

- 了解[可观测性](../../observability/)，以查看回调如何实现系统透明性
- 探索[回调集成](../../../../extras/callbacks/)，以获取现成的回调实现